In [15]:
from pulp import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
df = pd.read_csv("hns_062022.csv")
# df = df[df['event_date']>='2022-06-01']

In [29]:
rev = 'rev_14'

cost_li = []
revenue_li = []
install_li = []
gg_li  = []

for gg_cost in list(np.arange(500, 19000, 500)):
    min_project_install = 7

    total_install = int(df['install'].sum())
    total_cost = int(df['cost'].sum())
    min_install = total_install * 0.3
    max_cost = total_cost * 0.7

    grouped_df = df.groupby(['network', 'campaign', 'country_code'], as_index=False).agg({'install': 'sum', 'cost': 'sum', 'rev_14': 'sum', 'event_date': 'count', 'rev_30': 'sum'})

    grouped_df['is_adwords'] = grouped_df.apply(lambda r: 1 if r['network']=='adwords' else 0, axis=1)
    grouped_df['is_minter'] = grouped_df.apply(lambda r: 1 if r['network']=='mintegral' else 0, axis=1)
    grouped_df['is_unity'] = grouped_df.apply(lambda r: 1 if r['network']=='unity_ads' else 0, axis=1)

    grouped_df = grouped_df[(grouped_df['install']>=(min_project_install*grouped_df['event_date'])*grouped_df['is_adwords'])]
    grouped_df = grouped_df[(grouped_df['event_date'] >= 4*grouped_df['is_minter'])]
    grouped_df = grouped_df[grouped_df['event_date'] >= 6*grouped_df['is_unity']]

    grouped_df = grouped_df[(grouped_df['cost']) > 0]
    grouped_df = grouped_df[(grouped_df['install']) > 50]

    grouped_df = grouped_df.reset_index()
    grouped_df = grouped_df.drop('index', axis=1)

    list_index = list(grouped_df[grouped_df['network']=='adwords'].index)

    grouped_df['profit'] = grouped_df[rev] - grouped_df['cost']
    grouped_df['name'] = grouped_df['campaign'] + "-" + grouped_df['country_code'] + "-" + grouped_df['network']

    # 1. Initializa
    model = LpProblem("MKT_ROAS_small", LpMaximize)

    # 2. Define variables
    projects = list(grouped_df['name'].values)
    profit = list(grouped_df['profit'].values)

    # yes or no fund
    P = LpVariable.dicts('P', [j for j in range(len(projects))], lowBound=0, upBound=1, cat='Integer')

    # 3. Define Objective
    model += lpSum([profit[i] * P[i] for i in range(len(projects))])


    # 4. define constraints
    model += lpSum(P[i] * grouped_df.loc[i, 'cost'] for i in range(len(projects))) >= 0
    model += lpSum(P[i] * grouped_df.loc[i, 'cost'] for i in range(len(projects))) <= max_cost

    model += lpSum(P[i] * grouped_df.loc[i, 'cost'] for i in list_index) >= gg_cost
    
    # model += lpSum(P[i] * grouped_df.loc[i, 'install'] for i in range(len(projects))) >= min_install

    # for i in range(len(projects)):
    #     model += P[i] * grouped_df.loc[i, 'install'] >= 20

    status = model.solve()

    # Results
    ACTUAL_REV = round(sum([P[i].varValue * grouped_df.loc[i, rev] for i in range(len(projects))]),2)
    ACTUAL_BUDGET = round(sum([P[i].varValue * grouped_df.loc[i, 'cost'] for i in range(len(projects))]),2)
    ACTUAL_INSTALL = round(sum([P[i].varValue * grouped_df.loc[i, 'install'] for i in range(len(projects))]))
    # project_allocation = int(sum([P[i].varValue for i in range(len(projects))]))
    # LIST_PROJECTS = []
    # for j in range(len(projects)):
    #     LIST_PROJECTS.append(P[j].varValue)
    
    if LpStatus[status]=='Optimal':
        cost_li.append(ACTUAL_BUDGET)
        revenue_li.append(ACTUAL_REV)
        install_li.append(ACTUAL_INSTALL)
        gg_li.append(gg_cost)
    # print(LpStatus[status])
    # print("Total profit = {:,} USD (ROAS {}%)".format(
    #     round(value(model.objective), 2),
    #     round((ACTUAL_REV) / ACTUAL_BUDGET * 100,2)))
        
    # print('{}/{} Projects Accepted with a Budget Allocation of {:,} $ for {} install'.format(project_allocation, 
    #                                                                                 len(grouped_df), 
    #                                                                                 ACTUAL_BUDGET, 
    #                                                                                 ACTUAL_INSTALL))


In [31]:
pd.DataFrame.from_dict({'cost': cost_li, 'rev': revenue_li, 'install': install_li, 'gg_cost': gg_li})

,cost,rev,install,gg_cost
0,2548.13,4603.93,164221,500
1,2548.13,4603.93,164221,1000
2,3010.56,5021.33,173884,1500
3,3521.77,5467.05,177461,2000
4,4010.44,5890.72,191942,2500
5,4534.18,6339.74,222040,3000
6,5058.47,6780.08,252890,3500
7,5510.44,7138.49,273867,4000
8,6028.94,7551.72,283454,4500
9,6569.22,7941.27,278871,5000


In [33]:
rev = 'rev_14'
min_project_install = 7

total_install = int(df['install'].sum())
total_cost = int(df['cost'].sum())
min_install = total_install * 0.3
max_cost = total_cost * 0.7

grouped_df = df.groupby(['network', 'campaign', 'country_code'], as_index=False).agg({'install': 'sum', 'cost': 'sum', 'rev_14': 'sum', 'event_date': 'count', 'rev_30': 'sum'})

grouped_df['is_adwords'] = grouped_df.apply(lambda r: 1 if r['network']=='adwords' else 0, axis=1)
grouped_df['is_minter'] = grouped_df.apply(lambda r: 1 if r['network']=='mintegral' else 0, axis=1)
grouped_df['is_unity'] = grouped_df.apply(lambda r: 1 if r['network']=='unity_ads' else 0, axis=1)

grouped_df = grouped_df[(grouped_df['install']>=(min_project_install*grouped_df['event_date'])*grouped_df['is_adwords'])]
grouped_df = grouped_df[(grouped_df['event_date'] >= 4*grouped_df['is_minter'])]
grouped_df = grouped_df[grouped_df['event_date'] >= 6*grouped_df['is_unity']]

grouped_df = grouped_df[(grouped_df['cost']) > 0]
grouped_df = grouped_df[(grouped_df['install']) > 50]

grouped_df = grouped_df.reset_index()
grouped_df = grouped_df.drop('index', axis=1)

list_index = list(grouped_df[grouped_df['network']=='adwords'].index)

grouped_df['profit'] = grouped_df[rev] - grouped_df['cost']
grouped_df['name'] = grouped_df['campaign'] + "-" + grouped_df['country_code'] + "-" + grouped_df['network']

# 1. Initializa
model = LpProblem("MKT_ROAS_small", LpMaximize)

# 2. Define variables
projects = list(grouped_df['name'].values)
profit = list(grouped_df['profit'].values)

# yes or no fund
P = LpVariable.dicts('P', [j for j in range(len(projects))], lowBound=0, upBound=1, cat='Integer')

# 3. Define Objective
model += lpSum([profit[i] * P[i] for i in range(len(projects))])


# 4. define constraints
model += lpSum(P[i] * grouped_df.loc[i, 'cost'] for i in range(len(projects))) >= 0
model += lpSum(P[i] * grouped_df.loc[i, 'cost'] for i in range(len(projects))) <= max_cost

model += lpSum(P[i] * grouped_df.loc[i, 'cost'] for i in list_index) >= 3000

# model += lpSum(P[i] * grouped_df.loc[i, 'install'] for i in range(len(projects))) >= min_install

# for i in range(len(projects)):
#     model += P[i] * grouped_df.loc[i, 'install'] >= 20

status = model.solve()

# Results
ACTUAL_REV = round(sum([P[i].varValue * grouped_df.loc[i, rev] for i in range(len(projects))]),2)
ACTUAL_BUDGET = round(sum([P[i].varValue * grouped_df.loc[i, 'cost'] for i in range(len(projects))]),2)
ACTUAL_INSTALL = round(sum([P[i].varValue * grouped_df.loc[i, 'install'] for i in range(len(projects))]))
project_allocation = int(sum([P[i].varValue for i in range(len(projects))]))
LIST_PROJECTS = []
for j in range(len(projects)):
    LIST_PROJECTS.append(P[j].varValue)


print(LpStatus[status])
print("Total profit = {:,} USD (ROAS {}%)".format(
    round(value(model.objective), 2),
    round((ACTUAL_REV) / ACTUAL_BUDGET * 100,2)))
    
print('{}/{} Projects Accepted with a Budget Allocation of {:,} $ for {} install'.format(project_allocation, 
                                                                                len(grouped_df), 
                                                                                ACTUAL_BUDGET, 
                                                                                ACTUAL_INSTALL))


Optimal
Total profit = 1,805.56 USD (ROAS 139.82%)
105/128 Projects Accepted with a Budget Allocation of 4,534.18 $ for 222040 install


In [9]:
grouped_df['pick'] = LIST_PROJECTS
grouped_df.to_csv('hns_ROAS_40.csv')

In [260]:
grouped_df[grouped_df['network']=='adwords']['cost'].sum()

25659.619300000002

In [7]:
grouped_df[['install', 'cost', rev]].sum()

install    514461.0000
cost        27103.0329
rev_14      20431.7480
dtype: float64

In [229]:
df[['install', 'cost', rev]].sum()

install    389007.0000
cost        32555.7215
rev_14      32866.7079
dtype: float64

In [8]:
import plotly.express as px

In [247]:
new_df = df.groupby(['network', 'campaign', 'country_code'], as_index=False).agg({'install': 'sum', 'cost': 'sum', rev: 'sum', 'event_date': 'count', 'rev_30': 'sum'})
new_df.to_csv('mtd_ROAS_0.csv')

In [ ]:
fig_df = grouped_df[grouped_df['pick']==1].groupby(['network', 'campaign'], as_index=False)['profit'].sum()
fig_df['is_profit'] = fig_df['profit'].apply(lambda x: "Yes" if x>0 else "No")
fig_df = fig_df.groupby(['is_profit', 'network'], as_index=False)['campaign'].nunique()
fig = px.bar(fig_df, x='is_profit', y='campaign', color='network', text_auto='.2s')
fig.show()

In [10]:
fig_df = grouped_df[grouped_df['pick']==1].groupby(['network', 'campaign'], as_index=False)['profit'].sum()
fig_df['is_profit'] = fig_df['profit'].apply(lambda x: "Yes" if x>0 else "No")
fig_df = fig_df.groupby(['is_profit', 'network'], as_index=False)['campaign'].nunique()
fig = px.bar(fig_df, x='is_profit', y='campaign', color='network', text_auto='.2s')
fig.show()

In [11]:
fig_df = grouped_df.groupby(['network', 'campaign'], as_index=False)['profit'].sum()
fig_df['is_profit'] = fig_df['profit'].apply(lambda x: "Yes" if x>0 else "No")
fig_df = fig_df.groupby(['is_profit', 'network'], as_index=False)['campaign'].nunique()
fig = px.bar(fig_df, x='is_profit', y='campaign', color='network', text_auto='.2s')
fig.show()

In [12]:
fig_df = grouped_df[grouped_df['pick']==1].groupby(['network'], as_index=False)['cost'].sum()
# fig_df['cost'] = round(fig_df['cost'] / fig_df['cost'].sum() * 100, 1)
fig = px.bar(fig_df, x='network', y='cost', text_auto='.2s')
fig.show()

In [13]:
fig_df = df.groupby(['network'], as_index=False)['cost'].sum()
# fig_df['cost'] = round(fig_df['cost'] / fig_df['cost'].sum() * 100, 1)
fig = px.bar(fig_df, x='network', y='cost', text_auto='.2s')
fig.show()

In [30]:
fig_df = df.groupby(['campaign'], as_index=False)['install'].sum()

fig = px.histogram(fig_df, x="install", range_y=[0, 12])
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=64000.0,
        size=100
    ))
fig.show()

In [29]:
fig_df = grouped_df.groupby(['campaign'], as_index=False)['install'].sum()

fig = px.histogram(fig_df, x="install", range_y=[0, 12])
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=64000.0,
        size=100
    ))
fig.show()